# Sentiment Analysis with IMDB dataset using Pytorch

In [4]:
import numpy as np
import keras
from keras.datasets import imdb
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn
from keras.preprocessing.text import Tokenizer
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
%matplotlib inline

np.random.seed(42)

# Dataset Prepration

In [5]:
mean, std = (0.5,), (0.5,)

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test), = imdb.load_data(num_words=1000)
# restore np.load for future normal usage
np.load = np_load_old

print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


In [6]:
print(x_train[0])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


In [7]:
num_features = 1000
tokenizer = Tokenizer(num_words=num_features)
x_train  = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [8]:
print(x_train[0].shape)

(1000,)


In [9]:
num_classes = 2
y_train = np.eye(num_classes)[y_train.flatten()]
y_test = np.eye(num_classes)[y_test.flatten()]

In [10]:
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


In [12]:
x_train_batches, y_train_batches = np.split(x_train,500), np.split(y_train,500)

In [13]:
class Sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(num_features, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc2_ = nn.Dropout(p=0.7)
        self.fc3 = nn.Linear(128, 32)
        self.fc3_ = nn.Dropout(p=0.4)
        self.fc4 = nn.Linear(32, 4)
        self.fc4_ = nn.Dropout(p=0.2)
        self.fc5 = nn.Linear(4, num_classes)
    def forward(self, x):
        x = F.selu(self.fc1(x))
        x = self.fc2_(x)
        x = F.selu(self.fc3(x))
        x = self.fc3_(x)
        x = F.selu(self.fc4(x))
        x = self.fc4_(x)
        x = F.selu(self.fc5(x))
        x = F.sigmoid(x)

        return x

model = Sentiment()